In [199]:
from nba_api.stats.static import players,teams
from nba_api.stats.endpoints import PlayerVsPlayer, TeamVsPlayer, leaguegamefinder, playbyplay,CommonAllPlayers, CommonPlayerInfo
import pandas as pd 
import numpy as np
import json
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType

In [5]:
steph_id = players.find_players_by_full_name('Steph')[1]['id']
pat_id = players.find_players_by_full_name('Patrick')[0]['id']


In [3]:
PlayerVsTeam = TeamVsPlayer(team_id='1610612740', vs_player_id=steph_id).get_normalized_dict()

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

# how to access parameters of api calls

In [42]:
params = TeamVsPlayer(team_id='1610612740', vs_player_id=steph_id).parameters

# An example of TeamVsPlayer to DataFrame

In [44]:
normalized_dict = TeamVsPlayer(team_id='1610612740', vs_player_id=steph_id, season='2015-16').get_normalized_dict()
df = pd.DataFrame.from_dict(normalized_dict, orient = 'index')
TeamVsPlayer = df.transpose()
TeamVsPlayer['Overall']

TypeError: 'DataFrame' object is not callable

# An example of PlayerVsPlayer to DataFrame

In [28]:
data = PlayerVsPlayer(player_id = steph_id, vs_player_id = pat_id).get_normalized_dict()
df = pd.DataFrame.from_dict(data, orient = 'index')
PlayerVsPlayer = df.transpose()

In [13]:
pd.DataFrame.from_dict(df['ShotAreaOnCourt'][5],orient = 'index').transpose()

,GROUP_SET,PLAYER_ID,PLAYER_NAME,VS_PLAYER_ID,VS_PLAYER_NAME,COURT_STATUS,GROUP_VALUE,FGM,FGA,FG_PCT,CFID,CFPARAMS
0,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,Above the Break 3,0,7,0,100,201976


In [14]:
ShotAreaOnCourt_DF = pd.DataFrame(pd.DataFrame.from_dict(df['ShotAreaOnCourt'][0],orient = 'index').transpose())

In [16]:
# for i in df['ShotAreaOnCourt']:
#     ShotAreaOnCourt_DF.append(pd.DataFrame.from_dict(df['ShotAreaOnCourt'][i],orient = 'index').transpose())
    

In [17]:
pd.DataFrame.from_dict(df['ShotAreaOnCourt'][6],orient = 'index').transpose()

,GROUP_SET,PLAYER_ID,PLAYER_NAME,VS_PLAYER_ID,VS_PLAYER_NAME,COURT_STATUS,GROUP_VALUE,FGM,FGA,FG_PCT,CFID,CFPARAMS
0,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,Backcourt,0,0,0,101,201976


In [18]:
pd.concat([pd.DataFrame.from_dict(df['ShotAreaOnCourt'][i],orient = 'index').transpose() for i in range(7)],ignore_index=True)







###########

,GROUP_SET,PLAYER_ID,PLAYER_NAME,VS_PLAYER_ID,VS_PLAYER_NAME,COURT_STATUS,GROUP_VALUE,FGM,FGA,FG_PCT,CFID,CFPARAMS
0,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,Restricted Area,1,1,1,95,201976
1,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,In The Paint (Non-RA),1,2,0.5,96,201976
2,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,Mid-Range,1,2,0.5,97,201976
3,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,Left Corner 3,1,1,1,98,201976
4,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,Right Corner 3,0,0,0,99,201976
5,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,Above the Break 3,0,7,0,100,201976
6,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,Backcourt,0,0,0,101,201976


In [19]:
len(df['ShotAreaOnCourt'])

9

In [20]:
df.columns

Index(['Overall', 'OnOffCourt', 'ShotDistanceOverall', 'ShotDistanceOnCourt',
       'ShotDistanceOffCourt', 'ShotAreaOverall', 'ShotAreaOnCourt',
       'ShotAreaOffCourt', 'PlayerInfo', 'VsPlayerInfo'],
      dtype='object')

In [21]:
df['ShotAreaOverall'][0]

{'GROUP_SET': 'Shot Area',
 'GROUP_VALUE': 'Restricted Area',
 'PLAYER_ID': 201939,
 'PLAYER_NAME': 'Stephen Curry',
 'FGM': 11,
 'FGA': 16,
 'FG_PCT': 0.688,
 'CFID': 102,
 'CFPARAMS': None}

In [22]:
pd.DataFrame.from_dict(df['Overall'][1],orient = 'index').transpose()

,GROUP_SET,GROUP_VALUE,PLAYER_ID,PLAYER_NAME,GP,W,L,W_PCT,MIN,FGM,...,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,CFID,CFPARAMS
0,Overall,Patrick Beverley,201976,Patrick Beverley,7,5,2,0.714,211.395,18,...,12,2,1,23,21,43,38,156.4,85,Patrick Beverley


In [23]:
df['PlayerInfo']

0    {'PERSON_ID': 201939, 'FIRST_NAME': 'Stephen',...
1                                                 None
2                                                 None
3                                                 None
4                                                 None
5                                                 None
6                                                 None
7                                                 None
8                                                 None
Name: PlayerInfo, dtype: object

In [71]:
nba_teams = teams.get_teams()

# Select the dictionary for the Pacers, which contains their team ID
golden = [team for team in nba_teams if team['abbreviation'] == 'GSW'][0]
golden_id = golden['id']
print(f'pacers_id: {golden_id}')

pacers_id: 1610612744


In [72]:
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=golden_id,
                            season_nullable=Season.default,
                            season_type_nullable=SeasonType.regular)  

games_dict = gamefinder.get_normalized_dict()
games = games_dict['LeagueGameFinderResults']
game = games[0]
game_id = game['GAME_ID']
game_matchup = game['MATCHUP']

In [73]:
df = playbyplay.PlayByPlay(game_id).get_data_frames()[0]


In [1]:
headers = {
    'Host': 'stats.nba.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
}

In [7]:



trial = TeamVsPlayer(vs_player_id=steph_id,team_id='1610612737',season='2016-17',headers=headers).get_normalized_dict()

df = pd.DataFrame.from_dict(trial, orient = 'index').transpose()

In [72]:
df.iloc[2]['OnOffCourt']

,Overall,vsPlayerOverall,OnOffCourt,ShotDistanceOverall,ShotDistanceOnCourt,ShotDistanceOffCourt,ShotAreaOverall,ShotAreaOnCourt,ShotAreaOffCourt
0,"{'GROUP_SET': 'Overall', 'GROUP_VALUE': 'Atlan...","{'GROUP_SET': 'Overall', 'GROUP_VALUE': 'Steph...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Res...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
1,None,None,"{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'In ...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
2,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Mid...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
3,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Lef...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
4,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Rig...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
5,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Abo...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
6,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Bac...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
7,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...",None,"{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...",None,None,None
8,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...",None,None,None,None,None


In [76]:
nba_teams_info = pd.concat([pd.DataFrame.from_dict(teams.get_teams()[i], orient='index').transpose() for i in range(30)],ignore_index=True)


In [187]:
StephVsTeam_df = pd.DataFrame(columns = nba_teams_info['full_name'], index=['2018-19','2017-18','2016-17','2015-16','2014-15','2013-14','2012-13','2011-12','2010-11','2009-10'])


In [165]:
StephVsTeam_df

full_name,Atlanta Hawks,Boston Celtics,Cleveland Cavaliers,New Orleans Pelicans,Chicago Bulls,Dallas Mavericks,Denver Nuggets,Golden State Warriors,Houston Rockets,Los Angeles Clippers,...,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Oklahoma City Thunder,Toronto Raptors,Utah Jazz,Memphis Grizzlies,Washington Wizards,Detroit Pistons,Charlotte Hornets
2018-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
StephVsTeam_df.loc['2018-2019']['Atlanta Hawks']

nan

In [188]:
StephVsTeam_df.head()


full_name,Atlanta Hawks,Boston Celtics,Cleveland Cavaliers,New Orleans Pelicans,Chicago Bulls,Dallas Mavericks,Denver Nuggets,Golden State Warriors,Houston Rockets,Los Angeles Clippers,...,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Oklahoma City Thunder,Toronto Raptors,Utah Jazz,Memphis Grizzlies,Washington Wizards,Detroit Pistons,Charlotte Hornets
2018-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
nba_teams_info = pd.concat([pd.DataFrame.from_dict(teams.get_teams()[i], orient='index').transpose() for i in range(30)],ignore_index=True)

,ShotDistanceOnCourt,ShotDistanceOffCourt,ShotAreaOverall,ShotAreaOnCourt,ShotAreaOffCourt,PlayerInfo,VsPlayerInfo
0,"{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Res...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'PERSON_ID': 201939, 'FIRST_NAME': 'Stephen',...","{'PERSON_ID': 201976, 'FIRST_NAME': 'Patrick',..."
1,"{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'In ...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
2,"{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Mid...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
3,"{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Lef...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
4,"{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Rig...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
5,"{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Abo...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
6,"{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Bac...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
7,"{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None,None,None,None,None
8,None,None,None,None,None,None,None


In [258]:
def filter_col_team(df):
    return df.drop(['Overall','vsPlayerOverall','ShotDistanceOverall','ShotDistanceOffCourt','ShotAreaOverall','ShotAreaOffCourt'],axis=1)
    


In [259]:
def filter_col_player(df):
    return df.drop(['Overall','ShotDistanceOverall','ShotDistanceOffCourt','ShotAreaOverall','ShotAreaOffCourt'],axis=1)





In [ ]:
def season_team()

In [260]:
def insert_PlayerVsTeam(StephVsTeam_df,player_id,team_info):
    seasons = ['2018-19','2017-18','2016-17','2015-16','2014-15','2013-14','2012-13','2011-12','2010-11','2009-10']
    for i in seasons:
        team_id = team_info['id'][0]
        team_name = team_info['full_name'][0]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][1]
        team_name = team_info['full_name'][1]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][2]
        team_name = team_info['full_name'][2]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][3]
        team_name = team_info['full_name'][3]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][4]
        team_name = team_info['full_name'][4]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][5]
        team_name = team_info['full_name'][5]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][6]
        team_name = team_info['full_name'][6]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][7]
        team_name = team_info['full_name'][7]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][8]
        team_name = team_info['full_name'][8]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][9]
        team_name = team_info['full_name'][9]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][10]
        team_name = team_info['full_name'][10]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][11]
        team_name = team_info['full_name'][11]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][12]
        team_name = team_info['full_name'][12]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][13]
        team_name = team_info['full_name'][13]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][14]
        team_name = team_info['full_name'][14]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][15]
        team_name = team_info['full_name'][15]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][16]
        team_name = team_info['full_name'][16]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][17]
        team_name = team_info['full_name'][17]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][18]
        team_name = team_info['full_name'][18]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][19]
        team_name = team_info['full_name'][19]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][20]
        team_name = team_info['full_name'][20]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][21]
        team_name = team_info['full_name'][21]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][22]
        team_name = team_info['full_name'][22]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][23]
        team_name = team_info['full_name'][23]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][24]
        team_name = team_info['full_name'][24]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][25]
        team_name = team_info['full_name'][25]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][26]
        team_name = team_info['full_name'][26]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][27]
        team_name = team_info['full_name'][27]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][28]
        team_name = team_info['full_name'][28]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    for i in seasons:
        team_id = team_info['id'][29]
        team_name = team_info['full_name'][29]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    

In [269]:
StephVsTeam_df = insert_PlayerVsTeam(StephVsTeam_df,steph_id,nba_teams_info)

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [156]:
StephVsTeam_df.loc['2018-19']['Atlanta Hawks'] = df.iloc[5]['ShotAreaOnCourt']

In [134]:
nba_players_info = pd.concat([pd.DataFrame.from_dict(players.get_players()[i], orient='index').transpose() for i in range(len(players.get_players()))],ignore_index=True)

In [239]:
data = PlayerVsPlayer(player_id = steph_id, vs_player_id = pat_id,headers=headers,season='2015-16').get_normalized_dict()
df = pd.DataFrame.from_dict(data, orient = 'index').transpose()

In [237]:
df.iloc[1]['OnOffCourt']

{'GROUP_SET': 'Vs. Player',
 'PLAYER_ID': 201939,
 'PLAYER_NAME': 'Stephen Curry',
 'VS_PLAYER_ID': 201976,
 'VS_PLAYER_NAME': 'Beverley, Patrick',
 'COURT_STATUS': 'Off',
 'GP': 2,
 'W': 2,
 'L': 0,
 'W_PCT': 1.0,
 'MIN': 29.583333333333332,
 'FGM': 13,
 'FGA': 19,
 'FG_PCT': 0.684,
 'FG3M': 9,
 'FG3A': 14,
 'FG3_PCT': 0.643,
 'FTM': 0,
 'FTA': 0,
 'FT_PCT': 0.0,
 'OREB': 0,
 'DREB': 4,
 'REB': 4,
 'AST': 5,
 'TOV': 1,
 'STL': 1,
 'BLK': 0,
 'BLKA': 0,
 'PF': 4,
 'PFD': 0,
 'PTS': 35,
 'PLUS_MINUS': 26,
 'NBA_FANTASY_PTS': 49.2,
 'CFID': 87,
 'CFPARAMS': '201976'}

In [217]:
StephVsPlayers_df = pd.DataFrame(columns = players_active['full_name'], index=['2018-19','2017-18','2016-17','2015-16','2014-15','2013-14','2012-13','2011-12','2010-11','2009-10'])


In [222]:
players_active_array = players_active['full_name'].values

In [223]:
players_active_array

array(['Alex Abrines', 'Quincy Acy', 'Jaylen Adams', 'Steven Adams',
       'Bam Adebayo', 'Deng Adel', 'DeVaughn Akoon-Purcell',
       'LaMarcus Aldridge', 'Rawle Alkins', 'Grayson Allen',
       'Jarrett Allen', 'Kadeem Allen', 'Al-Farouq Aminu',
       'Justin Anderson', 'Kyle Anderson', 'Ryan Anderson',
       'Ike Anigbogu', 'Giannis Antetokounmpo', 'Kostas Antetokounmpo',
       'Carmelo Anthony', 'OG Anunoby', 'Ryan Arcidiacono',
       'Trevor Ariza', 'D.J. Augustin', 'Deandre Ayton', 'Dwayne Bacon',
       'Marvin Bagley III', 'Ron Baker', 'Wade Baldwin IV', 'Lonzo Ball',
       'Mo Bamba', 'J.J. Barea', 'Harrison Barnes', 'Will Barton',
       'Keita Bates-Diop', 'Nicolas Batum', 'Jerryd Bayless',
       'Aron Baynes', 'Kent Bazemore', 'Bradley Beal', 'Malik Beasley',
       'Michael Beasley', 'Marco Belinelli', 'Jordan Bell',
       "DeAndre' Bembry", 'Dragan Bender', 'Dairis Bertans',
       'Davis Bertans', 'Patrick Beverley', 'Justin Bibbs', 'Khem Birch',
       'Bismack

full_name,Alex Abrines,Quincy Acy,Jaylen Adams,Steven Adams,Bam Adebayo,Deng Adel,DeVaughn Akoon-Purcell,LaMarcus Aldridge,Rawle Alkins,Grayson Allen,...,Delon Wright,Guerschon Yabusele,Nick Young,Thaddeus Young,Trae Young,Cody Zeller,Tyler Zeller,Zhou Qi,Ante Zizic,Ivica Zubac
2018-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [261]:
def insert_PlayerVsPlayer(StephVsPlayers_df,player_id,players_active):
    for (i,j) in zip(players_active['full_name'],players_active['id']):
        dict_matchup = PlayerVsPlayer(player_id=player_id, vs_player_id=j, headers=headers,season='2018-19').get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_matchup, orient = 'index').transpose()
        filtered_col = filter_col_player(dict_to_df)
        StephVsPlayers_df.loc['2018-19'][i] = filtered_col
    for (i,j) in zip(players_active['full_name'],players_active['id']):
        dict_matchup = PlayerVsPlayer(player_id=player_id, vs_player_id=j, headers=headers,season='2017-18').get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_matchup, orient = 'index').transpose()
        filtered_col = filter_col_player(dict_to_df)
        StephVsPlayers_df.loc['2017-18'][i] = filtered_col
    for (i,j) in zip(players_active['full_name'],players_active['id']):
        dict_matchup = PlayerVsPlayer(player_id=player_id, vs_player_id=j, headers=headers,season='2016-17').get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_matchup, orient = 'index').transpose()
        filtered_col = filter_col_player(dict_to_df)
        StephVsPlayers_df.loc['2016-17'][i] = filtered_col
    for (i,j) in zip(players_active['full_name'],players_active['id']):
        dict_matchup = PlayerVsPlayer(player_id=player_id, vs_player_id=j, headers=headers,season='2015-16').get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_matchup, orient = 'index').transpose()
        filtered_col = filter_col_player(dict_to_df)
        StephVsPlayers_df.loc['2015-16'][i] = filtered_col
    for (i,j) in zip(players_active['full_name'],players_active['id']):
        dict_matchup = PlayerVsPlayer(player_id=player_id, vs_player_id=j, headers=headers,season='2014-15').get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_matchup, orient = 'index').transpose()
        filtered_col = filter_col_player(dict_to_df)
        StephVsPlayers_df.loc['2014-15'][i] = filtered_col
    for (i,j) in zip(players_active['full_name'],players_active['id']):
        dict_matchup = PlayerVsPlayer(player_id=player_id, vs_player_id=j, headers=headers,season='2013-14').get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_matchup, orient = 'index').transpose()
        filtered_col = filter_col_player(dict_to_df)
        StephVsPlayers_df.loc['2013-14'][i] = filtered_col
    for (i,j) in zip(players_active['full_name'],players_active['id']):
        dict_matchup = PlayerVsPlayer(player_id=player_id, vs_player_id=j, headers=headers,season='2012-13').get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_matchup, orient = 'index').transpose()
        filtered_col = filter_col_player(dict_to_df)
        StephVsPlayers_df.loc['2012-13'][i] = filtered_col
    for (i,j) in zip(players_active['full_name'],players_active['id']):
        dict_matchup = PlayerVsPlayer(player_id=player_id, vs_player_id=j, headers=headers,season='2011-12').get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_matchup, orient = 'index').transpose()
        filtered_col = filter_col_player(dict_to_df)
        StephVsPlayers_df.loc['2011-12'][i] = filtered_col
    for (i,j) in zip(players_active['full_name'],players_active['id']):
        dict_matchup = PlayerVsPlayer(player_id=player_id, vs_player_id=j, headers=headers,season='2010-11').get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_matchup, orient = 'index').transpose()
        filtered_col = filter_col_player(dict_to_df)
        StephVsPlayers_df.loc['2010-11'][i] = filtered_col
    for (i,j) in zip(players_active['full_name'],players_active['id']):
        dict_matchup = PlayerVsPlayer(player_id=player_id, vs_player_id=j, headers=headers,season='2009-10').get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_matchup, orient = 'index').transpose()
        filtered_col = filter_col_player(dict_to_df)
        StephVsPlayers_df.loc['2009-10'][i] = filtered_col
    

In [262]:
players_active = nba_players_info[nba_players_info['is_active'] == True].reset_index(drop=True)
StephVsPlayers_df = insert_PlayerVsPlayer(StephVsPlayers_df,steph_id,players_active)

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [267]:
StephVsPlayers_df.loc['2017-18']['Patrick Beverley']

,OnOffCourt,ShotDistanceOnCourt,ShotAreaOnCourt,PlayerInfo,VsPlayerInfo
0,"{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'PERSON_ID': 201939, 'FIRST_NAME': 'Stephen',...","{'PERSON_ID': 201976, 'FIRST_NAME': 'Patrick',..."
1,"{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
2,None,"{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
3,None,"{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
4,None,"{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
5,None,"{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
6,None,"{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
7,None,None,None,None,None
8,None,None,None,None,None


In [281]:
def season_team(season,player_id,nba_team_info):
    new_df = pd.DataFrame(columns=nba_team_info['full_name'],index = [season])
    for (i,j) in zip(nba_team_info['full_name'],nba_team_info['id']):
        dict_team = TeamVsPlayer(team_id=j,season=season,vs_player_id=player_id, headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col_team(dict_to_df)
        new_df.loc[season][i] = filtered_col
    return new_df

        

In [282]:
test = season_team("2018-19",steph_id,nba_teams_info)

In [289]:
test2 = season_team("2017-18",steph_id,nba_teams_info)

In [292]:
test3 = season_team("2016-17",steph_id,nba_teams_info)

In [291]:
test.append(test2)

full_name,Atlanta Hawks,Boston Celtics,Cleveland Cavaliers,New Orleans Pelicans,Chicago Bulls,Dallas Mavericks,Denver Nuggets,Golden State Warriors,Houston Rockets,Los Angeles Clippers,...,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Oklahoma City Thunder,Toronto Raptors,Utah Jazz,Memphis Grizzlies,Washington Wizards,Detroit Pistons,Charlotte Hornets
2018-19,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...
2017-18,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...


In [297]:
pd.concat([test,test2,test3],axis = 0)

full_name,Atlanta Hawks,Boston Celtics,Cleveland Cavaliers,New Orleans Pelicans,Chicago Bulls,Dallas Mavericks,Denver Nuggets,Golden State Warriors,Houston Rockets,Los Angeles Clippers,...,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Oklahoma City Thunder,Toronto Raptors,Utah Jazz,Memphis Grizzlies,Washington Wizards,Detroit Pistons,Charlotte Hornets
2018-19,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...
2017-18,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...
2016-17,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...,OnOf...


In [298]:
test4 = season_team("2015-16",steph_id,nba_teams_info)

KeyboardInterrupt: 

In [299]:
steph_id

201939

In [300]:
nba_teams_info

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970
